# Direct Solvers

The purpose of this notebook is to compare commonly available direct solvers from python interface. Here we focus on multi-threaded solvers that can be run on a single node with several cores.
For now, we compare SuperLU, UMFPACK and Intel MKL PARDISO. SuperLU is the default solver in scipy, UMFPACK requires the installation of scikit-umfpack and Intel MKL PARDISO requires PyPardiso.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg as spla

import ibmos as ib

In [3]:
import scipy.linalg as la

# Case initialization
... from [Unsteady flow around cylinder (Re=200)](../1-Basic/CylinderRe200.ipynb)

In [4]:
s1 = ib.stretching(256, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
s2 = ib.stretching(128, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
x = np.r_[-s2[::-1], s1[1:]]

s = ib.stretching(192, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
y = np.r_[-s[::-1], s[1:]]

solver = ib.Solver(x, y, iRe=1/200, Co=0.4)
del x, y, s1, s2

solver.set_solids(ib.shapes.cylinder("cylinder", 0, 0, 0.5, solver.dxmin))

uBC, vBC = solver.zero_boundary_conditions()
for k in range(4):
    uBC[k][:] = 1

sBC = ((np.zeros(solver.solids[0].l), np.zeros(solver.solids[0].l)), )

In [5]:
n = 50

# Basic tests

In [6]:
A, B = solver.propagator(fractionalStep=False)
bc = solver.boundary_condition_terms(uBC, vBC, *sBC)

### Factorization

In [7]:
%%time 
splu, =ib.tools.solver_superlu(A[0].copy())
x=splu(bc)
x[solver.pStart:solver.pEnd]-=np.mean(x[solver.pStart:solver.pEnd])
print(la.norm(A[0]@x-bc)/la.norm(bc))

1.1608912400598384e-11
CPU times: user 9min 15s, sys: 12.9 s, total: 9min 27s
Wall time: 29.5 s


In [8]:
%%time 
umfp, = ib.tools.solver_umfpack(A[0].copy())
x=umfp(bc)
x[solver.pStart:solver.pEnd]-=np.mean(x[solver.pStart:solver.pEnd])
print(la.norm(A[0]@x-bc)/la.norm(bc))

3.689651412028829e-12
CPU times: user 2min 27s, sys: 4.19 s, total: 2min 32s
Wall time: 6.09 s


/home/miguel/.local/lib/python3.6/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.73e+17)
  warnings.warn(msg, UmfpackWarning)


In [9]:
%%time 
pdiso, pdisoSolver = ib.tools.solver_pardiso(A[0].copy())
pdisoSolver.set_statistical_info_on()
x=pdiso(bc); 
x[solver.pStart:solver.pEnd]-=np.mean(x[solver.pStart:solver.pEnd])
print(la.norm(A[0]@x-bc)/la.norm(bc))

3.654045793433579e-12
CPU times: user 15.5 s, sys: 1.41 s, total: 16.9 s
Wall time: 1.8 s


### Solve

In [10]:
%%timeit -n 10
splu(bc);

266 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit -n 10
umfp(bc);

/home/miguel/.local/lib/python3.6/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.73e+17)
  warnings.warn(msg, UmfpackWarning)


546 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit -n 10
pdiso(bc);

74.4 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# SuperLU

In [13]:
solver.set_solver(ib.tools.solver_superlu)

## Without fractional step method

In [14]:
solver.set_fractional_step(False)

In [15]:
%time x0, = solver.steps(solver.zero(), uBC, vBC, sBC, number=1, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) 
     1    0.013200 1.000000e+00 264.414954210  0.000000000 1.042087e-11
CPU times: user 9min 9s, sys: 12.8 s, total: 9min 22s
Wall time: 29 s


In [16]:
%time x, = solver.steps(x0, uBC, vBC, sBC, number=n, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) 
     1    0.013200 1.111026e+03  2.138647404 -0.000000000 9.641530e-16
     2    0.026400 4.325909e-02  2.017210217 -0.000000000 8.963994e-16
     3    0.039600 4.131016e-02  1.886824089  0.000000000 9.034220e-16
     4    0.052800 2.890206e-02  1.773961242 -0.000000000 9.020607e-16
     5    0.066000 1.521691e-02  1.678152406  0.000000000 7.962614e-16
     6    0.079200 7.493112e-03  1.597628520 -0.000000000 8.757517e-16
     7    0.092400 6.050111e-03  1.530030261 -0.000000000 9.203459e-16
     8    0.105600 7.289223e-03  1.472723456  0.000000000 9.025657e-16
     9    0.118800 7.669492e-03  1.423404027 -0.000000000 9.044582e-16
    10    0.132000 6.001224e-03  1.380496281 -0.000000000 9.748553e-16
    11    0.145200 3.539894e-03  1.343108465  0.000000000 8.882065e-16
    12    0.158400 2.005103e-03  1.310663100  0.000000000 7.985786e-16
    13    0.171600 1.728156e-03  1.282556972 -0.000000000 8.947622e-16
    1

## With fractional step method

In [17]:
solver.set_fractional_step(True)

In [18]:
%time x0, = solver.steps(solver.zero(), uBC, vBC, sBC, number=1, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) rel.error(C) 
     1    0.013200 1.000000e+00 264.406914312 -0.000000000 1.459429e-16 6.622222e-12
CPU times: user 8min 59s, sys: 11.3 s, total: 9min 11s
Wall time: 24 s


In [19]:
%time x, = solver.steps(x0, uBC, vBC, sBC, number=n, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) rel.error(C) 
     1    0.013200 1.111194e+03  2.139664108  0.000000000 2.112804e-16 1.423370e-14
     2    0.026400 4.310738e-02  2.018073432 -0.000000000 2.110005e-16 1.275840e-14
     3    0.039600 4.119458e-02  1.887486028  0.000000000 2.113227e-16 1.152514e-14
     4    0.052800 2.882995e-02  1.774466838  0.000000000 2.108925e-16 9.893072e-15
     5    0.066000 1.517764e-02  1.678540792  0.000000000 2.114102e-16 9.681488e-15
     6    0.079200 7.470886e-03  1.597928410  0.000000000 2.107655e-16 9.432308e-15
     7    0.092400 6.031451e-03  1.530263063  0.000000000 2.114780e-16 9.429830e-15
     8    0.105600 7.270344e-03  1.472905831  0.000000000 2.108466e-16 9.257602e-15
     9    0.118800 7.653639e-03  1.423548904  0.000000000 2.115505e-16 9.511891e-15
    10    0.132000 5.991208e-03  1.380613071  0.000000000 2.114998e-16 9.120277e-15
    11    0.145200 3.534673e-03  1.343203601  0.000000000 2.111709e-16 9.41

# UMFPACK

In [20]:
solver.set_solver(ib.tools.solver_umfpack)

## Without fracional step method

In [21]:
solver.set_fractional_step(False)

In [22]:
%time x0, = solver.steps(solver.zero(), uBC, vBC, sBC, number=1, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) 
     1    0.013200 1.000000e+00 264.414954210  0.000000000 3.301895e-12
CPU times: user 2min 34s, sys: 4.34 s, total: 2min 38s
Wall time: 6.33 s


/home/miguel/.local/lib/python3.6/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.73e+17)
  warnings.warn(msg, UmfpackWarning)


In [23]:
%time x, = solver.steps(x0, uBC, vBC, sBC, number=n, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) 


/home/miguel/.local/lib/python3.6/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.73e+17)
  warnings.warn(msg, UmfpackWarning)


     1    0.013200 1.111026e+03  2.138647404 -0.000000000 8.448525e-16
     2    0.026400 4.325909e-02  2.017210217  0.000000000 1.512334e-15
     3    0.039600 4.131016e-02  1.886824089  0.000000000 1.717606e-15
     4    0.052800 2.890206e-02  1.773961242 -0.000000000 1.768356e-15
     5    0.066000 1.521691e-02  1.678152406  0.000000000 1.551488e-15
     6    0.079200 7.493112e-03  1.597628520 -0.000000000 9.538657e-16
     7    0.092400 6.050111e-03  1.530030261  0.000000000 7.401091e-16
     8    0.105600 7.289223e-03  1.472723456 -0.000000000 4.420478e-16
     9    0.118800 7.669492e-03  1.423404027  0.000000000 1.230816e-15
    10    0.132000 6.001224e-03  1.380496281 -0.000000000 1.460788e-15
    11    0.145200 3.539894e-03  1.343108465  0.000000000 7.402059e-16
    12    0.158400 2.005103e-03  1.310663100  0.000000000 1.499355e-15
    13    0.171600 1.728156e-03  1.282556972 -0.000000000 1.446633e-15
    14    0.184800 2.019596e-03  1.258089635 -0.000000000 1.146404e-15
    15

## With fractional step method

In [24]:
solver.set_fractional_step(True)

In [25]:
%time x0, = solver.steps(solver.zero(), uBC, vBC, sBC, number=1, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) rel.error(C) 
     1    0.013200 1.000000e+00 264.406914320  0.000000000 1.191361e-16 2.711905e-11
CPU times: user 2min 2s, sys: 3.35 s, total: 2min 5s
Wall time: 5.21 s


/home/miguel/.local/lib/python3.6/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 8.39e+12)
  warnings.warn(msg, UmfpackWarning)


In [26]:
%time x, = solver.steps(x0, uBC, vBC, sBC, number=n, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) rel.error(C) 


/home/miguel/.local/lib/python3.6/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 8.39e+12)
  warnings.warn(msg, UmfpackWarning)


     1    0.013200 1.111194e+03  2.139664100 -0.000000000 1.206488e-16 1.777476e-14
     2    0.026400 4.310738e-02  2.018073432 -0.000000000 1.206556e-16 1.600722e-14
     3    0.039600 4.119458e-02  1.887486028  0.000000000 1.202722e-16 1.498895e-14
     4    0.052800 2.882995e-02  1.774466838 -0.000000000 1.204948e-16 1.357754e-14
     5    0.066000 1.517764e-02  1.678540792 -0.000000000 1.205676e-16 1.372565e-14
     6    0.079200 7.470886e-03  1.597928410 -0.000000000 1.202356e-16 1.302290e-14
     7    0.092400 6.031451e-03  1.530263063 -0.000000000 1.209381e-16 1.254913e-14
     8    0.105600 7.270344e-03  1.472905831 -0.000000000 1.210358e-16 1.341947e-14
     9    0.118800 7.653639e-03  1.423548904 -0.000000000 1.207710e-16 1.268749e-14
    10    0.132000 5.991208e-03  1.380613071 -0.000000000 1.206900e-16 1.272898e-14
    11    0.145200 3.534673e-03  1.343203601 -0.000000000 1.207202e-16 1.244618e-14
    12    0.158400 2.001652e-03  1.310741100 -0.000000000 1.204598e-16 1.320

# PARDISO

In [27]:
solver.set_solver(ib.tools.solver_pardiso)

## Without fractional step method

In [28]:
solver.set_fractional_step(False)

In [29]:
%time x0, = solver.steps(solver.zero(), uBC, vBC, sBC, number=1, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) 
     1    0.013200 1.000000e+00 264.414954210  0.000000000 3.287320e-12
CPU times: user 24.6 s, sys: 1.54 s, total: 26.2 s
Wall time: 2.1 s


In [30]:
%time x, = solver.steps(x0, uBC, vBC, sBC, number=n, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) 
     1    0.013200 1.111026e+03  2.138647404 -0.000000000 1.493367e-16
     2    0.026400 4.325909e-02  2.017210217 -0.000000000 1.488799e-16
     3    0.039600 4.131016e-02  1.886824089 -0.000000000 1.487625e-16
     4    0.052800 2.890206e-02  1.773961242 -0.000000000 1.498452e-16
     5    0.066000 1.521691e-02  1.678152406 -0.000000000 1.489727e-16
     6    0.079200 7.493112e-03  1.597628520 -0.000000000 1.492305e-16
     7    0.092400 6.050111e-03  1.530030261 -0.000000000 1.496564e-16
     8    0.105600 7.289223e-03  1.472723456 -0.000000000 1.490987e-16
     9    0.118800 7.669492e-03  1.423404027 -0.000000000 1.486409e-16
    10    0.132000 6.001224e-03  1.380496281 -0.000000000 1.495932e-16
    11    0.145200 3.539894e-03  1.343108465 -0.000000000 1.494988e-16
    12    0.158400 2.005103e-03  1.310663100 -0.000000000 1.497596e-16
    13    0.171600 1.728156e-03  1.282556972 -0.000000000 1.487519e-16
    1

## With fractional step method

In [31]:
solver.set_fractional_step(True)

In [32]:
%time x0, = solver.steps(solver.zero(), uBC, vBC, sBC, number=1, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) rel.error(C) 
     1    0.013200 1.000000e+00 264.406914318 -0.000000000 1.491906e-16 1.185919e-11
CPU times: user 44.7 s, sys: 2.68 s, total: 47.4 s
Wall time: 2.68 s


In [33]:
%time x, = solver.steps(x0, uBC, vBC, sBC, number=n, reportEvery=1, verbose=True)

  step      t        residual   cylinder(fx) cylinder(fy) rel.error(A) rel.error(C) 
     1    0.013200 1.111194e+03  2.139664102  0.000000000 1.895341e-16 1.247720e-14
     2    0.026400 4.310738e-02  2.018073432  0.000000000 1.906954e-16 9.870800e-15
     3    0.039600 4.119458e-02  1.887486028  0.000000000 1.899551e-16 8.458484e-15
     4    0.052800 2.882995e-02  1.774466838  0.000000000 1.905652e-16 9.190644e-15
     5    0.066000 1.517764e-02  1.678540792  0.000000000 1.900261e-16 7.162256e-15
     6    0.079200 7.470886e-03  1.597928410  0.000000000 1.899558e-16 8.831938e-15
     7    0.092400 6.031451e-03  1.530263063  0.000000000 1.892200e-16 6.454688e-15
     8    0.105600 7.270344e-03  1.472905831  0.000000000 1.907117e-16 7.246681e-15
     9    0.118800 7.653639e-03  1.423548904  0.000000000 1.894385e-16 8.161934e-15
    10    0.132000 5.991208e-03  1.380613071  0.000000000 1.901621e-16 7.607339e-15
    11    0.145200 3.534673e-03  1.343203601  0.000000000 1.896980e-16 7.31